In [13]:
import pandas as pd
import os, re

df = pd.read_csv('data/clean_data_final.csv')
df.head()

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,USM,01-01-2021 01:00,True
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,USM,01-01-2021 02:00,True
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,USM,01-01-2021 03:00,True
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565,USM,01-01-2021 04:00,True
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,USM,01-01-2021 05:00,True


Read the information related to the stations 

In [14]:
# Cambio
stations = pd.read_csv('data/stations_loc.csv')
stations = stations[['Sigla', 'estacion', 'Localidad', 'Latitud', 'Longitud']]
stations = stations.rename(columns={'Sigla': 'Station'})
stations = stations.rename(columns={'estacion': 'Name'})
stations.head()

,Station,Name,Localidad,Latitud,Longitud
0,GYR,guaymaral,Suba,"4°47'01.5""N","74°02'38.9""W"
1,USQ,usaquen,Usaquén,"4°42'37.26""N","74°1'49.50""W"
2,SUB,suba,Suba,"4°45'40.49""N","74° 5'36.46""W"
3,BOL,bolivia,Engativá,"4°44'08.9""N","74°07'33.2""W"
4,LFR,las_ferias,Engativá,"4°41'26.52""N","74°4'56.94""W"


Convert the coordinates string to decimal

In [15]:
# TODO: move to utils.py
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(coor):
    parts = re.split('[^\d\w]+', coor)
    dec_coor = dms2dd(parts[0], parts[1], float(parts[2]+'.'+parts[2]), parts[4])

    return dec_coor

Convert the columns

In [16]:
stations['Latitud'] = stations['Latitud'].apply(parse_dms)
stations['Longitud'] = stations['Longitud'].apply(parse_dms)
stations.head()

,Station,Name,Localidad,Latitud,Longitud
0,GYR,guaymaral,Suba,4.783614,-74.043994
1,USQ,usaquen,Usaquén,4.710381,-74.030414
2,SUB,suba,Suba,4.761222,-74.093433
3,BOL,bolivia,Engativá,4.735578,-74.125925
4,LFR,las_ferias,Engativá,4.690628,-74.082378


Add the geolocation to each entry

In [17]:
df = pd.merge(df, stations, on='Station', how='inner')
df

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Name,Localidad,Latitud,Longitud
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,USM,01-01-2021 01:00,True,usame,Usme,4.532097,-74.116947
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,USM,01-01-2021 02:00,True,usame,Usme,4.532097,-74.116947
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,USM,01-01-2021 03:00,True,usame,Usme,4.532097,-74.116947
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565,USM,01-01-2021 04:00,True,usame,Usme,4.532097,-74.116947
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,USM,01-01-2021 05:00,True,usame,Usme,4.532097,-74.116947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166435,43.1,12.0,11.771,23.697,35.468,0.98779,8.146,KEN,31-12-2021 20:00,False,kennedy,Kennedy,4.625083,-74.161222
166436,35.0,35.0,6.634,21.677,28.311,0.85772,8.055,KEN,31-12-2021 21:00,True,kennedy,Kennedy,4.625083,-74.161222
166437,43.5,43.0,2.666,19.867,22.533,0.84339,6.877,KEN,31-12-2021 22:00,True,kennedy,Kennedy,4.625083,-74.161222
166438,60.1,55.0,2.622,19.689,22.311,0.84038,6.533,KEN,31-12-2021 23:00,True,kennedy,Kennedy,4.625083,-74.161222


Save the data to a new file

In [18]:
df.to_csv("data/clean_data_final_geo.csv", index = False)

In [19]:
df.head()

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Name,Localidad,Latitud,Longitud
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,USM,01-01-2021 01:00,True,usame,Usme,4.532097,-74.116947
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,USM,01-01-2021 02:00,True,usame,Usme,4.532097,-74.116947
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,USM,01-01-2021 03:00,True,usame,Usme,4.532097,-74.116947
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565,USM,01-01-2021 04:00,True,usame,Usme,4.532097,-74.116947
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,USM,01-01-2021 05:00,True,usame,Usme,4.532097,-74.116947
